In [1]:
import os
import datetime

import pandas as pd
import numpy as np

In [2]:
path_raw_data = './data/raw_data/'
file_list = os.listdir(path_raw_data)
len(file_list)

9

In [3]:
file_list

['kr_regional_daily.csv',
 '산둥지역 일일 미세먼지.csv',
 '상하이종합 내역_일간.csv',
 '상하이종합 내역_주간.csv',
 '서울 날씨.xlsx',
 '서울 미세먼지 농도(pm2.5,pm10).xlsx',
 '서울시 교통량.xlsx',
 '중국 일일 코로나 확진자 수.xlsx',
 '코스피지수.xlsx']

In [4]:
seoul_covid = pd.read_csv(path_raw_data+file_list[0])
china_sandoung_dust = pd.read_csv(path_raw_data+file_list[1])
china_stock_daily = pd.read_csv(path_raw_data+file_list[2])
weather = pd.read_excel(path_raw_data+file_list[4])
y = pd.read_excel(path_raw_data+file_list[5])
transport = pd.read_excel(path_raw_data+file_list[6])
china_covid = pd.read_excel(path_raw_data+file_list[7])
kospi =  pd.read_excel(path_raw_data+file_list[8])

# 필요 데이터만

seoul_covid 

china_sandoung_dust

china_stock_daily

weather 

y 

transport

china_covid

kospi

In [5]:
seoul_covid = seoul_covid[seoul_covid.region =='서울']
seoul_covid = seoul_covid[['date', 'confirmed', 'death', 'released']]

In [6]:
china_stock_daily = china_stock_daily[['날짜', '종가']]
china_stock_daily['종가'] = china_stock_daily['종가'].apply(lambda cnt: float(cnt.replace(',','')))

In [7]:
weather = weather.fillna(0)

In [8]:
china_covid = china_covid[china_covid.countriesAndTerritories == 'China']
china_covid = china_covid[['dateRep','cases', 'deaths']]

In [9]:
seoul_covid.columns = ['date', 'seoul_covid_confirmed', 'seoul_covid_death', 'seoul_covid_released']
china_sandoung_dust.columns = ['date', 'china_pm2.5', 'china_pm10']
china_stock_daily.columns = ['date', 'china_stock_index']
weather.columns = ['date', '기온', '강수량', '풍속', '최다풍향', '습도', '기압', '안개시간']
y.columns = ['date', 'y_pm2.5', 'y_pm10']
transport.columns = ['date', 'transport_in', 'transport_out', 'transport_inandout']
china_covid.columns = ['date', 'china_covid_confirmed', 'china_covid_death']
kospi.columns = ['date', 'kospi_index']

In [10]:
seoul_covid['date'] = pd.to_datetime(seoul_covid.date, format='%Y%m%d')
china_sandoung_dust['date'] = pd.to_datetime(china_sandoung_dust.date, format='%Y-%m-%d')
china_stock_daily['date'] = pd.to_datetime(china_stock_daily.date, format='%Y년 %m월 %d일')
weather['date'] = pd.to_datetime(weather.date, format='%Y%m%d')
y['date'] = pd.to_datetime(y.date, format='%Y%m%d')
transport['date'] = pd.to_datetime(transport.date, format='%Y-%m-%d')
china_covid['date'] = pd.to_datetime(china_covid.date, format='china_covid')
kospi['date'] = pd.to_datetime(kospi.date, format='%Y%m%d')

In [11]:
seoul_covid = seoul_covid.set_index('date')
china_sandoung_dust = china_sandoung_dust.set_index('date')
china_stock_daily = china_stock_daily.set_index('date')
weather = weather.set_index('date')
y = y.set_index('date')
transport = transport.set_index('date')
china_covid = china_covid.set_index('date')
kospi = kospi.set_index('date')

In [12]:
merge_data = pd.concat([seoul_covid, china_sandoung_dust, china_stock_daily, weather, y, transport, china_covid, kospi], axis=1)

In [13]:
merge_data[['seoul_covid_confirmed', 'seoul_covid_death', 'seoul_covid_released',
            'china_covid_confirmed', 'china_covid_death']] = merge_data[['seoul_covid_confirmed', 'seoul_covid_death', 'seoul_covid_released',
                                                                         'china_covid_confirmed', 'china_covid_death']].fillna(0)

In [14]:
merge_data = merge_data[merge_data.index <  datetime.datetime(2020, 11, 1)]

In [15]:
merge_data.to_csv('./data/first_merge.csv', encoding='cp949')

# 추가 데이터

In [16]:
add_data = pd.read_csv('./data/add_data/서울 석유소비.csv').drop(['월', '서 울', '합 계'], axis=1)

In [17]:
add_data.columns = ['date', 'soeul_oil_consumption']

In [18]:
add_data['date'] = pd.to_datetime(add_data.date.apply(lambda cnt: int('20'+str(cnt))), format='%Y%m')

In [19]:
add_data = add_data.set_index('date')

In [20]:
final_merge_data = pd.concat([merge_data, add_data], axis=1)

In [21]:
final_merge_data[['soeul_oil_consumption']] = final_merge_data[['soeul_oil_consumption']].fillna(method='bfill')

In [22]:
final_merge_data = final_merge_data.fillna(method='pad')

In [23]:
final_merge_data.to_csv('./data/final_merge.csv', encoding='cp949')

# 간단 EDA

In [24]:
from pandas_profiling import ProfileReport

In [25]:
profile = ProfileReport(final_merge_data, explorative=True)

In [26]:
profile.to_file("./docs/output.html")

C:\Users\User\anaconda3\lib\site-packages\matplotlib\textpath.py:74: RuntimeWarning: Glyph 44592 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
C:\Users\User\anaconda3\lib\site-packages\matplotlib\textpath.py:74: RuntimeWarning: Glyph 50728 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
C:\Users\User\anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 44592 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\User\anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 50728 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\User\anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py:201: RuntimeWarning: Glyph 44592 missing from current font.
  font.set_text(s, 0, flags=flags)
C:\Users\User\anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py:201: RuntimeWarning: Glyph 50728 missing from current font.
 

C:\Users\User\anaconda3\lib\site-packages\matplotlib\textpath.py:74: RuntimeWarning: Glyph 50504 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
C:\Users\User\anaconda3\lib\site-packages\matplotlib\textpath.py:74: RuntimeWarning: Glyph 44060 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
C:\Users\User\anaconda3\lib\site-packages\matplotlib\textpath.py:74: RuntimeWarning: Glyph 49884 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
C:\Users\User\anaconda3\lib\site-packages\matplotlib\textpath.py:74: RuntimeWarning: Glyph 44036 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
C:\Users\User\anaconda3\lib\site-packages\matplotlib\textpath.py:74: RuntimeWarning: Glyph 44592 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
C:\Users\User\anaconda3\lib\site-packages\matplotlib\textpath.py:74: RuntimeWarning: Glyph 50517 missing from current font.
  font.set_text(s, 0.

C:\Users\User\anaconda3\lib\site-packages\pandas_profiling\visualisation\plot.py:154: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. In future versions, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = copy.copy(mpl.cm.get_cmap("RdBu"))
  cmap.set_bad(cmap_bad)
